In [ ]:


import datetime, os
from breeze_connect import BreezeConnect
# Retrieve credentials from environment variables\n"
api_key = os.getenv("BREEZE_API_KEY")
secret_key = os.getenv("BREEZE_SECRET_KEY")
customer_id = os.getenv("BREEZE_CUSTOMER_ID")
session_token = os.getenv("BREEZE_SESSION_TOKEN")
breeze = BreezeConnect(api_key=api_key)

# Login and get session key
import urllib
print("https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus(api_key))



In [ ]:
# Generate Session

breeze.generate_session(api_secret=secret_key,
                        session_token=session_token)



In [ ]:
from datetime import datetime, timedelta
import time
import bisect
from collections import defaultdict
import pickle
import os
STRIKE_PRICE = 'Strike Price'
CALL_LTP = 'CALL_LTP'
PUT_LTP = 'PUT_LTP'
GAMMA_CALL = 'GAMMA_CALL'
nifty_call_data = defaultdict(dict)
def add_to_stock(strike_price, sample_data, type):
    for data in sample_data:
        date = datetime.strptime(data['datetime'],"%Y-%m-%d %H:%M:%S")
        nifty_call_data[date][strike_price]={CALL_LTP: data['close']}
expDates= [
        "05-Jan-2023",
        "12-Jan-2023",
        "19-Jan-2023",
        "25-Jan-2023",
        "02-Feb-2023",
        "09-Feb-2023",
        "16-Feb-2023",
        "23-Feb-2023",
        "02-Mar-2023",
        "09-Mar-2023",
        "16-Mar-2023",
        "23-Mar-2023",
        "29-Mar-2023",
        "06-Apr-2023",
        "13-Apr-2023",
        "20-Apr-2023",
        "27-Apr-2023",
        "04-May-2023",
        "11-May-2023",
        "18-May-2023",
        "25-May-2023",
        "01-Jun-2023",
        "08-Jun-2023",
        "15-Jun-2023",
        "22-Jun-2023",
        "28-Jun-2023",
        "29-Jun-2023",
        "06-Jul-2023",
        "13-Jul-2023",
        "20-Jul-2023",
        "27-Jul-2023",
        "03-Aug-2023",
        "10-Aug-2023",
        "17-Aug-2023",
        "24-Aug-2023",
        "31-Aug-2023",
        "07-Sep-2023",
        "14-Sep-2023",
        "21-Sep-2023",
        "28-Sep-2023",
        "05-Oct-2023",
        "12-Oct-2023",
        "19-Oct-2023",
        "26-Oct-2023",
        "02-Nov-2023",
        "09-Nov-2023",
        "16-Nov-2023",
        "23-Nov-2023",
        "30-Nov-2023",
        "07-Dec-2023",
        "14-Dec-2023",
        "21-Dec-2023",
        "28-Dec-2023",
        "04-Jan-2024",
        "11-Jan-2024",
        "18-Jan-2024",
        "25-Jan-2024",
        "01-Feb-2024",
        "08-Feb-2024",
        "15-Feb-2024",
        "22-Feb-2024",
        "29-Feb-2024",
        "07-Mar-2024",
        "14-Mar-2024",
        "21-Mar-2024",
        "28-Mar-2024",
        "04-Apr-2024",
        "10-Apr-2024",
        "18-Apr-2024",
        "25-Apr-2024",
        "02-May-2024",
        "09-May-2024",
        "16-May-2024",
        "23-May-2024",
        "30-May-2024",
        "06-Jun-2024",
        "13-Jun-2024",
        "27-Jun-2024",
        "25-Jul-2024",
        "26-Sep-2024",
        "26-Dec-2024"
    ]
date_format = "%d-%b-%Y"
expiry_dates = [datetime.strptime(f,date_format) for f in expDates]
def getExpiry(date):
    if type(date) == str:
        parsed_date = datetime.strptime(date, date_format)
    else:
        parsed_date = date
    next_exp = bisect.bisect_left(expiry_dates, parsed_date)
    if len(expiry_dates)-1 <= next_exp:
      return expiry_dates[-1]
    return expiry_dates[next_exp]

def getExpiry2(date):
    if type(date) == str:
        parsed_date = datetime.strptime(date, date_format)
    else:
        parsed_date = date
    for x in expiry_dates:
        if x >= parsed_date :
            return x
    return x

def get_historical_data(interval, from_date, to_date, stock_code, exchange_code, product_type, right, strike_price, all_data):


    # Strings to datetime objects
    from_date = datetime.strptime(from_date, "%Y-%m-%d")
    to_date = datetime.strptime(to_date, "%Y-%m-%d")
    start = from_date.strftime("%Y-%m-%d")
    end = to_date.strftime("%Y-%m-%d")
    filename = f'{right}_{start}_{end}_{exchange_code}_{strike_price}.pickle'
    if os.path.exists(filename):
        with open(filename, 'rb') as data:
            all_data = pickle.load(data)
        return all_data
    #days=365*3
    time_range = timedelta(days=3)

    # Loop until the from_date exceeds the to_date
    while from_date <= to_date:
        if from_date.weekday == 5 or from_date.weekday==6 :
            from_date += timedelta(days=1)
            continue
        start_exp = getExpiry(from_date)
        till_date = from_date+timedelta(days=3)
        end_exp = getExpiry(till_date)

        if start_exp != end_exp:
            till_date = start_exp
        from_date = from_date.replace(hour=9, minute=16)
        till_date = till_date.replace(hour=15, minute=29)
        print(from_date.strftime("%Y-%m-%dT%H:%M:%S.%fZ").replace('000000Z', '000Z'))
        # print(till_date.strftime("%Y-%m-%dT%H:%M:%S.%fZ").replace('000000Z','000Z'))

        data = breeze.get_historical_data_v2(
            interval=interval,
            from_date=from_date.strftime("%Y-%m-%dT%H:%M:%S.%fZ").replace('000000Z', '000Z'),
            to_date=till_date.strftime("%Y-%m-%dT%H:%M:%S.%fZ").replace('000000Z', '000Z'),
            stock_code=stock_code,
            exchange_code=exchange_code,
            product_type=product_type,
            expiry_date=start_exp,
            right=right,
            strike_price=strike_price
        )
        meta = {
            STRIKE_PRICE : strike_price,
            "type": product_type,
            "start_date": from_date,
            "end_date": till_date,
            "data": data,
        }

        all_data.append(meta)
        from_date = till_date + timedelta(days=1)
        time.sleep(0.5)  # API Sleep

        # if to_date.year == 2023: # Break if year is 2023
        #     break
        # print(all_data)

    with open(filename,'wb') as data:
        pickle.dump(all_data,data)
    return all_data


In [ ]:
start_price = 18000

while start_price < 23000 :

  all_data= []
  kk=get_historical_data(interval="1minute",
                              from_date= "2024-04-01",
                              to_date= "2024-05-17",
                              stock_code="NIFTY",
                              exchange_code="NSE",
                              product_type="options",
                              right="call",
                              strike_price=str(start_price),
                              all_data=all_data) # outputs 1000 clicks
  start_price += 50
  print("cooling down for 10 seconds")
  time.sleep(10)